In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
from google.colab import files
uploaded = files.upload()  

Saving FakeRE.csv to FakeRE.csv
Saving TrueRE.csv to TrueRE.csv


In [ ]:
import io

In [ ]:
# load the data
true = pd.read_csv(io.BytesIO(uploaded['TrueRE.csv']))
fake = pd.read_csv(io.BytesIO(uploaded['FakeRE.csv']))

In [ ]:
fake['fakeOrNot'] = 'FAKE'
fake.head()

,TITLE,TEXT,SUBJECT,DATE,fakeOrNot
0,Teen Records Himself Abusing Girl: ‘This Is W...,A young man recorded himself as he maliciously...,News,"January 3, 2016",FAKE
1,Professional Panderer Carly Fiorina Says Ted ...,Just days after forsaking her Stanford Cardina...,News,"January 3, 2016",FAKE
2,Poll Finds White Republicans Are Angry Nearly...,In a new poll that asked Americans to comment ...,News,"January 3, 2016",FAKE
3,"What’s Happening In Oregon Is TREASON, Our Fo...","What s happening in Oregon isn t, by any means...",News,"January 3, 2016",FAKE
4,Here Are The Right-Wing Scumbags The Bundy Mi...,When the tense standoff between federal agents...,News,"January 3, 2016",FAKE


In [ ]:
true['fakeOrNot'] = 'TRUE'
true.head()

,TITLE,TEXT,SUBJECT,DATE,fakeOrNot
0,Trump budget opens new fight among Republicans,WASHINGTON (Reuters) - Republican U.S. Represe...,politicsNews,"March 13, 2017",TRUE
1,Trump tried to call New York prosecutor Bharar...,WASHINGTON (Reuters) - Two days before U.S. At...,politicsNews,"March 12, 2017",TRUE
2,NY prosecutor’s office likely to plot similar ...,NEW YORK/WASHINGTON (Reuters) - The Trump admi...,politicsNews,"March 13, 2017",TRUE
3,Puerto Rico creditors urge extension of fiscal...,(Reuters) - A group holding more than $10 bill...,politicsNews,"March 13, 2017",TRUE
4,Biden says would have liked to be the U.S. pre...,"AUSTIN, Texas (Reuters) - Former Democratic U....",politicsNews,"March 12, 2017",TRUE


In [ ]:
# Concatenate Real and Fake News
df = pd.concat([true, fake]).reset_index(drop = True)
df

,TITLE,TEXT,SUBJECT,DATE,fakeOrNot
0,Trump budget opens new fight among Republicans,WASHINGTON (Reuters) - Republican U.S. Represe...,politicsNews,"March 13, 2017",TRUE
1,Trump tried to call New York prosecutor Bharar...,WASHINGTON (Reuters) - Two days before U.S. At...,politicsNews,"March 12, 2017",TRUE
2,NY prosecutor’s office likely to plot similar ...,NEW YORK/WASHINGTON (Reuters) - The Trump admi...,politicsNews,"March 13, 2017",TRUE
3,Puerto Rico creditors urge extension of fiscal...,(Reuters) - A group holding more than $10 bill...,politicsNews,"March 13, 2017",TRUE
4,Biden says would have liked to be the U.S. pre...,"AUSTIN, Texas (Reuters) - Former Democratic U....",politicsNews,"March 12, 2017",TRUE
...,...,...,...,...,...
11974,Liberal Imperium: Quigley’s Anglo-American Est...,Jay Dyer 21st Century WireThis is an addendum ...,Middle-east,"August 25, 2017",FAKE
11975,"SYRIA, NORTH KOREA: Trump’s “Axis of Evil” is ...",Whitney Webb Mint PressBush s Axis of Evil s...,Middle-east,"August 25, 2017",FAKE
11976,AFGHANISTAN: Trump Surges Into the Graveyard o...,"By ramping up US troop levels in Afghanistan,...",Middle-east,"August 24, 2017",FAKE
11977,Black Politicians Increase Attacks on Ben Cars...,Politics can be a nasty business. This latest ...,Middle-east,"August 23, 2017",FAKE


In [ ]:
df.drop(columns = ['DATE'], inplace = True)

In [ ]:
df.drop(columns = ['SUBJECT'], inplace = True)

In [ ]:
# combine title and text together
df['ORIGINAL'] = df['TITLE'] + ' ' + df['TEXT']
df.head()

,TITLE,TEXT,fakeOrNot,ORIGINAL
0,Trump budget opens new fight among Republicans,WASHINGTON (Reuters) - Republican U.S. Represe...,TRUE,Trump budget opens new fight among Republicans...
1,Trump tried to call New York prosecutor Bharar...,WASHINGTON (Reuters) - Two days before U.S. At...,TRUE,Trump tried to call New York prosecutor Bharar...
2,NY prosecutor’s office likely to plot similar ...,NEW YORK/WASHINGTON (Reuters) - The Trump admi...,TRUE,NY prosecutor’s office likely to plot similar ...
3,Puerto Rico creditors urge extension of fiscal...,(Reuters) - A group holding more than $10 bill...,TRUE,Puerto Rico creditors urge extension of fiscal...
4,Biden says would have liked to be the U.S. pre...,"AUSTIN, Texas (Reuters) - Former Democratic U....",TRUE,Biden says would have liked to be the U.S. pre...


In [ ]:
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
labels = df.fakeOrNot

In [ ]:
labels.value_counts()

TRUE    5999
FAKE    5980
Name: fakeOrNot, dtype: int64

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df['ORIGINAL'], labels, test_size=0.2, random_state=7)

In [ ]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [ ]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [ ]:
#Prediction on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 98.16%


In [ ]:
# Actual vs Predicted 
dframe = pd.DataFrame(y_test,y_pred)
dfr=pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
dfr

,Actual,Predicted
5203,TRUE,TRUE
10879,FAKE,FAKE
9039,FAKE,FAKE
7809,FAKE,FAKE
1334,TRUE,TRUE
...,...,...
7297,FAKE,FAKE
10416,FAKE,FAKE
5638,TRUE,TRUE
4471,TRUE,TRUE


In [ ]:
#Saving model
import pickle
pickle_out = open("fakenews.pkl","wb")
pickle.dump(pac,pickle_out)
pickle_out.close()

In [ ]:
from google.colab import files
files.download('fakenews.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>